<a href="https://colab.research.google.com/github/BaileyDalton007/discord_rnn/blob/main/discord_rnn_word_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
from gensim.models.word2vec import Word2Vec as w2v

from sklearn.decomposition import PCA
from matplotlib import pyplot as plt

import math
import csv
import pandas as pd
import numpy as np

import plotly.express as go

# Matplotlib outputs tons of warning for missing characters
import warnings
warnings.filterwarnings('ignore')

from google.colab import files

In [ ]:
# Upload training csv's
uploaded = files.upload()

In [4]:
file_names = list(uploaded.keys())

In [5]:
from os import linesep
# Create master array for messages from all files
msg_array = []

for file in file_names:
  with open(file, 'r', encoding='utf8') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')

    for row in csv_reader:
      msg_array.append(row[0].lower())

# Tokenize msg_array
msg_array = [sub.split() for sub in msg_array]

In [ ]:
# If model already trained and saved, load it.
model_save_file = files.upload()
model = w2v.load('word2vec.model')

In [ ]:
# Train word2vec model, may take a bit
model = w2v(msg_array, size=100, min_count=2, window=5, iter=100)

print(model)

In [ ]:
model.save('word2vec.model')
files.download('word2vec.model')

In [8]:
vocab = list(model.wv.vocab)

In [62]:
# Find frequency of words
word_count = {}
for word in vocab:
  word_count[word] = 0

for sequence in msg_array:
  for word in sequence:
    if word in word_count:
      word_count[word] += 1

In [ ]:
print(model.wv.__getitem__('car'))

In [ ]:
# This is horrendous please don't

X = model.wv.__getitem__(model.wv.vocab)
pca = PCA(n_components=2)

result = pca.fit_transform(X)

# Create a scatter plot of the projection
plt.scatter(result[:, 0], result[:, 1])
words = list(model.wv.vocab)

for i, word in enumerate(words):
   plt.annotate(word, xy=(result[i, 0], result[i, 1]))

plt.show()

In [ ]:
# Pass the embeddings to PCA
X = model.wv.__getitem__(model.wv.vocab)
pca = PCA(n_components=2)
result = pca.fit_transform(X)

# Create df from the pca results
pca_df = pd.DataFrame(result, columns = ['x','y'])

# Add the words for the hover effect
pca_df['word'] = vocab
pca_df.head()

In [100]:
# Formatting for plotly to play nice
graph_df = pd.DataFrame()

word_counts = list(word_count.values())

graph_df['word'] = pca_df['word']
graph_df['x'] = pca_df['x']
graph_df['y'] = pca_df['y']
graph_df['count'] = word_counts
graph_df['text'] = 'Word: ' + graph_df['word'] + " | Uses: " + graph_df['count'].apply(str)

# Scaled frequency as it was being skewed for very common words
graph_df['freq'] = [math.log(num) for num in word_counts]

In [ ]:
fig = go.scatter(
  graph_df,
  x='x',
  y='y',
  color='freq',
  text='text',
  title="Word2Vec Model Visualized with PCA"
)

fig.update_traces(mode="markers")
fig.show()

In [102]:
# Save interactive plot
fig.write_html("w2v_plot.html")

In [ ]:
model.wv.most_similar('car')

In [ ]:
model.wv.most_similar_cosmul(positive = ['best', 'player'])